## Upload all of the initially parsed data to huggingface

In [ ]:
import json

def normalize_mongo_json(data):
    cleaned = []
    for item in data:
        obj = {}

        for k, v in item.items():
            if k in ["_id", "__v", "created_at"]:
                continue  # drop fields

            # unwrap MongoDB extended JSON types
            if isinstance(v, dict) and "$date" in v:
                # ensure it's ISO 8601 without Z
                dt = v["$date"]
                # remove Z if present
                dt = dt.rstrip("Z")
                obj[k] = dt
            else:
                obj[k] = v

        cleaned.append(obj)

    return cleaned


# Example usage:
raw_json = json.loads(open("raw_dump_stage_1.json").read())
clean_json = normalize_mongo_json(raw_json)

with open("parsed_dump_stage_1.json", "w") as f:
    json.dump(clean_json, f, indent=2)

raw_json = json.loads(open("raw_dump_stage_2.json").read())
clean_json = normalize_mongo_json(raw_json)

with open("parsed_dump_stage_2.json", "w") as f:
    json.dump(clean_json, f, indent=2)


In [24]:
from datasets import Dataset

stage_1_dataset = Dataset.from_json("parsed_dump_stage_1.json")
stage_2_dataset = Dataset.from_json("parsed_dump_stage_2.json")
print(f"{len(stage_1_dataset)} {len(stage_2_dataset)}")

201583 247820


In [ ]:
import json
from datasets import load_dataset, Dataset, DatasetDict
dataset_1 = Dataset.from_json("parsed_dump_stage_1.json")
dataset_2 = Dataset.from_json("parsed_dump_stage_2.json")

dataset_1 = dataset_1.map(
    lambda x: {
        "relevant": False,   # default boolean
        "sentiment": 0       # default integer (-1, 0, 1 allowed)
    }
)
dataset_2 = dataset_2.map(
    lambda x: {
        "relevant": False,  # default boolean
        "sentiment": 0,  # default integer (-1, 0, 1 allowed)
    }
)

Map:   0%|          | 0/201583 [00:00<?, ? examples/s]

Map:   0%|          | 0/247820 [00:00<?, ? examples/s]

In [ ]:
dataset_dict = DatasetDict()
dataset_dict["source_stage_1"] = dataset_1
dataset_dict["source_stage_2"] = dataset_2
dataset_dict.push_to_hub("tianharjuno/twitter-parse", commit_message="Initial Commit")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/619 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/dbf93f69b6513a82701a40b76a6e91b8a84dc3b6', commit_message='Initial Commit', commit_description='', oid='dbf93f69b6513a82701a40b76a6e91b8a84dc3b6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)

In [28]:
from datasets import load_dataset
ds = load_dataset("tianharjuno/twitter-parse", cache_dir="cache/")
raw_ds = ds["source_stage_2"]

README.md:   0%|          | 0.00/776 [00:00<?, ?B/s]

data/source_stage_1-00000-of-00001.parqu(…):   0%|          | 0.00/24.5M [00:00<?, ?B/s]

data/source_stage_2-00000-of-00001.parqu(…):   0%|          | 0.00/30.2M [00:00<?, ?B/s]

Generating source_stage_1 split:   0%|          | 0/201583 [00:00<?, ? examples/s]

Generating source_stage_2 split:   0%|          | 0/247820 [00:00<?, ? examples/s]

In [30]:
import re, unicodedata, jaconv, emoji

_URL      = re.compile(r'https?://\S+')
_MENTION  = re.compile(r'@\w+')
_REPEAT   = re.compile(r'(.)\1{2,}')       # ≥3 of same char
_WS       = re.compile(r'\s+')
_KUTI_CUT = re.compile(r'(?i)kutipan.*$', re.DOTALL)

# --- (MODIFIED) ---
# Catches "word" + "dari" + "domain.com" -> replaces with "word"
# Changed \w+ to \S+ to include punctuation like '!'
_DARI_URL_ATTACHED = re.compile(r'(\S+)dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I) 

# Catches " dari " + "domain.com" -> replaces with empty string
_DARI_URL_SPACED = re.compile(r'\s+dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I)

# Catches any standalone word that looks like a domain.com or domain.id
_DOMAIN_ONLY = re.compile(r'\b[a-z0-9.-]+\.[a-z]{2,}\b', re.I)

# --- (NEW) ---
# Catches any word ending in "dari" (e.g., "anarko!dari", "negaradari")
_DARI_STUCK = re.compile(r'(\S+)dari\b', re.I)

def cleantext(row: str):
    text = row["content"] #type: ignore
    text = unicodedata.normalize('NFKC', text)
    text = jaconv.z2h(text, kana=False, digit=True, ascii=True)
    text = text.replace("tanya grok", " ")
    text = text.replace("grokproductivitypasang", " ")
    text = text.replace('\\n', ' ').replace('\\r', ' ')
    
    # Handle standard URLs first
    text = _URL.sub(' <url> ', text)
    text = text.replace('<url> ini tidak tersedia', ' ')
    
    # --- (TYPO FIXED) ---
    text = _MENTION.sub('@USER', text)
    text = re.sub(r'^rt\s+', '', text, flags=re.I)
    text = re.sub(r'(\b\d{4})(?=[a-zA-Z])', r'\1 ', text)
    text = _KUTI_CUT.sub('', text)

    # --- RULE ORDER IS IMPORTANT ---
    # 1. Fix "word-dari domain.com" first
    text = _DARI_URL_ATTACHED.sub(r'\1', text) 
    # 2. Fix " word dari domain.com"
    text = _DARI_URL_SPACED.sub('', text)
    
    # --- (ADD THIS LINE) ---
    # 3. Fix any remaining "word-dari"
    text = _DARI_STUCK.sub(r'\1', text)
    # --- END OF ADDITION ---
    
    # 4. Clean up any other standalone domains
    text = _DOMAIN_ONLY.sub(' ', text)

    text = emoji.demojize(text, delimiters=(' ', ' '))
    text = _REPEAT.sub(r'\1', text)
    text = _WS.sub(' ', text).strip().lower()
    row["content"] = text #type: ignore
    return row

In [31]:
raw_ds = raw_ds.map(cleantext)

Map:   0%|          | 0/247820 [00:00<?, ? examples/s]

In [32]:
content_list = raw_ds["content"]

# Correct syntax: "\n".join(content_list)
# Added encoding="utf-8" to prevent errors with special characters
with open("reader.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(content_list))

In [33]:
ds["cleaned"] = raw_ds

In [34]:
ds.push_to_hub("tianharjuno/twitter-parse", commit_description="cleaned up text")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/ca036224dd3ee9a5d9f7b4f4113c9a8995a76c8b', commit_message='Upload dataset', commit_description='cleaned up text', oid='ca036224dd3ee9a5d9f7b4f4113c9a8995a76c8b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)

In [41]:
train_ds = ds["cleaned"]
train_ds_split = train_ds.train_test_split(train_size=5000, seed=42)
test_ds = train_ds_split["train"]
leftover_ds = train_ds_split["test"]

In [44]:
leftover_ds_split = leftover_ds.train_test_split(train_size=20000, seed=42)
final_train_ds = leftover_ds_split["train"]
final_train_ds

Dataset({
    features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
    num_rows: 20000
})

In [45]:
ds

DatasetDict({
    source_stage_1: Dataset({
        features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
        num_rows: 201583
    })
    source_stage_2: Dataset({
        features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
        num_rows: 247820
    })
    cleaned: Dataset({
        features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
        num_rows: 247820
    })
})

In [46]:
ds["test"] = test_ds
ds["train"] = final_train_ds

ds.push_to_hub("tianharjuno/twitter-parse", commit_description="Created a test dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/887 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/b4688dfa70fcf6ae9ed10434e1a1c3cbe9202f2b', commit_message='Upload dataset', commit_description='Created a test dataset', oid='b4688dfa70fcf6ae9ed10434e1a1c3cbe9202f2b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)